In [1]:
!pip install -q torch                                  # Pytorch
!pip install -q transformers datasets                  # Comes from HuggingFace
!pip install -q bitsandbytes                           # For quantization from HuggingFace
!pip install -q peft                                   # Parameter-efficient Fine-tuning from HuggingFace
!pip install -q trl                                    # For supervised fine-tuning for LLMs from HuggingFace
!pip install -q accelerate        

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()
my_hf_key= os.getenv('HF_KEY')

login(token= my_hf_key)

In [3]:
import pandas as pd

df_C = pd.read_csv("deliverables/df_csv/df_C.csv", encoding='utf-8')
df_C.shape

(2800, 2)

In [4]:
df_C.head()

,German,French
0,Meine Freundin arbeitet im Büro und ich besuch...,Ma petite amie travaille au bureau et je la vi...
1,Herr Direktor wunscht Ihnen alles Gute und sch...,Le directeur vous souhaite une santé de fer et...
2,"Das Internet bietet viele Möglichkeiten, um si...",Internet offre de nombreuses possibilités pour...
3,"Mein Chef ist sehr anspruchsvoll, aber ich res...","Mon patron est très exigeant, mais je respecte..."
4,Das Lesen von Büchern über verschiedene Kultur...,La lecture de livres sur différentes cultures ...


We will use all these examples for training. But we will split 5% of them for validation.

In [5]:
from sklearn.model_selection import train_test_split

train_data, validation_data = train_test_split(df_C, test_size=0.1, random_state=42)
print(f"items in training and validation  are: {len(train_data)} and  {len(validation_data)}")



items in training and validation  are: 2520 and  280


In [6]:
from datasets import Dataset

# Convert the DataFrames into a Dataset
train_dataset = Dataset.from_pandas(train_data)
validation_dataset = Dataset.from_pandas(validation_data)

In [7]:
train_dataset

Dataset({
    features: ['German', 'French', '__index_level_0__'],
    num_rows: 2520
})

In [8]:
def format_instruction(german_input, french_trans):
    return f"""### Instruction:Translate to French.
### Input German Sentence:
{german_input.strip()}
### French Translation:
{french_trans.strip()}<|endoftext|>""" 
#EOS token for Qwen 

In [9]:
# Define a function that converts a dataset row into the corresponding prompt format
def convert_to_instruction_format(data_point):
    return {
        "text": format_instruction(data_point["German"], data_point["French"])
        }

In [10]:
# Apply the function to each row in the dataset
def process_dataset(data):
    return data.map(
        convert_to_instruction_format
        ).remove_columns(['German', 'French', '__index_level_0__']) #removing unnecessary columns

In [11]:
train_data = process_dataset(train_dataset)
validation_data = process_dataset(validation_dataset)

Map:   0%|          | 0/2520 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

In [12]:
train_data

Dataset({
    features: ['text'],
    num_rows: 2520
})

#### Load the model and finetune

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                             device_map = "auto",torch_dtype="auto")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [14]:
# Set the padding token to be the same as the end-of-sequence token.

# Padding ensures all sequences in a batch are of equal length.
tokenizer.pad_token = tokenizer.eos_token

# Specify that padding should be applied to the right side of the sequences.
# This is the standard behavior for causal language models. Causal models, predict the next token in a sequence using only the preceding tokens.
tokenizer.padding_side = "right"

In [15]:
# Import necessary PEFT objects for preparing the model for LoRA training
from peft import  LoraConfig, get_peft_model


model.enable_input_require_grads()


#model = prepare_model_for_kbit_training(model)  
#Use only if loading a quantized model-includes float conversions to help stabilize training

# Set up LoRA configuration
lora_config = LoraConfig(
    r=16,                                                       # The rank (dimensions) of the LoRA matrices A and B
    lora_alpha=64,                                              # Scales the product of matrices AB [W_new = W_old + (A * B) * α]
    target_modules=["q_proj", "v_proj"],    # Apply LoRA to the attention matrices
    lora_dropout=0.1,                                           # Dropout rate to reduce overfitting
    bias="none",                                                # Do not train the bias parameter
    task_type="CAUSAL_LM"                                       # Task type for autoregressive text generation
)

# Get the model with unfrozen LoRA layers applied
model = get_peft_model(model, lora_config)

In [16]:
from trl import SFTConfig

# Set up the training hyperparameters
training_arguments = SFTConfig(
    fp16=True,                           # Use 16-bit precision for training computations (optimizer states, gradients)
    dataset_text_field="text",           # Specify the text field in the dataset for training
    max_seq_length=128,                 # Set the maximum sequence length for the training data

    # Batch-related parameters
    per_device_train_batch_size=4,       # Batch size per device during training

    # Optimizer-related parameters
    optim="paged_adamw_32bit",           # Use the paged AdamW optimizer, optimized for 32-bit GPUs
    learning_rate=1e-4,                  # Set the learning rate for training

    # Epochs and saving configuration
    num_train_epochs=4,                  # Number of training epochs (more epochs generally lead to better results)
    save_strategy="epoch",               # Save the model after each epoch
    output_dir="./epoch-finetuned",      # Directory to save the fine-tuned model

    # Validation-related parameters
    eval_strategy="steps",               # Evaluation strategy, performed at specified steps
    eval_steps=0.2,                      # Evaluate after 20% of the training steps

    # Logging-related parameters
    report_to="none",                    # Disable reporting to external tools
    logging_dir="./logs",                # Directory to save the training logs
    logging_steps=20,                    # Number of steps between each log entry
    seed=42,                             # Set a random seed for reproducibility
)

# Enable gradient checkpointing to save memory and recompute during backpropagation
model.gradient_checkpointing_enable()

# Disable attention cache during training; it should be enabled during inference
model.config.use_cache = False

In [17]:
# Import the SFTTrainer from HuggingFace TRL library
from trl import SFTTrainer

# Initialize the trainer
trainer = SFTTrainer(
    # Assign the model and tokenizer
    model=model,
    tokenizer=tokenizer,

    # Provide the training and validation datasets
    train_dataset= train_data,
    eval_dataset=validation_data,

    # Pass the LoRA configuration
    peft_config=lora_config,

    # Set the training hyperparameters
    args=training_arguments,
)

/tmp/ipykernel_75540/2900952727.py:5: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/2520 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2520 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2520 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2520 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/280 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/280 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/280 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/280 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [18]:
# Start the training process 

trainer.train()

Step,Training Loss,Validation Loss
504,0.935200,0.978554
1008,0.908200,0.947416
1512,0.798000,0.928107
2016,0.803600,0.923929
2520,0.756800,0.919810


TrainOutput(global_step=2520, training_loss=0.8796610998728919, metrics={'train_runtime': 652.9848, 'train_samples_per_second': 15.437, 'train_steps_per_second': 3.859, 'total_flos': 4769581015388160.0, 'train_loss': 0.8796610998728919})

In [19]:
# Define the save path for the fine-tuned model on Colab
peft_model_path = "./fine-tuned-qwen2.5-1.5b-instruct-step10"

# Save the trained model
trainer.model.save_pretrained(peft_model_path)

# Save the tokenizer
tokenizer.save_pretrained(peft_model_path)

# List the saved files
!ls -lh {peft_model_path}

total 24M
-rwxr--r-- 1 prasanna99h prasanna99h 5.0K Mar 12 15:36 README.md
-rwxr--r-- 1 prasanna99h prasanna99h  722 Mar 12 15:36 adapter_config.json
-rwxr--r-- 1 prasanna99h prasanna99h 8.4M Mar 12 15:36 adapter_model.safetensors
-rwxr--r-- 1 prasanna99h prasanna99h  605 Mar 12 15:36 added_tokens.json
-rwxr--r-- 1 prasanna99h prasanna99h 1.6M Mar 12 15:36 merges.txt
-rwxr--r-- 1 prasanna99h prasanna99h  496 Mar 12 15:36 special_tokens_map.json
-rwxr--r-- 1 prasanna99h prasanna99h  11M Mar 12 15:36 tokenizer.json
-rwxr--r-- 1 prasanna99h prasanna99h 7.2K Mar 12 15:36 tokenizer_config.json
-rwxr--r-- 1 prasanna99h prasanna99h 2.7M Mar 12 15:36 vocab.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Evaluation

#### Before loading the tuned model, lets move back the original model to cpu. We can also delete the model

In [20]:
import torch

model.to("cpu")  # Move the existing model to CPU

torch.cuda.empty_cache()  # Clear GPU cache

In [21]:
# For loading a PEFT model, we need to use a special object for CausalLM from PEFT
# instead of the regular HuggingFace object.
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

# Load the fine-tuned model
peft_model_path = "./fine-tuned-qwen2.5-1.5b-instruct-step10"
tuned_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_path,
      # Load with 4-bit quantization
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tuned_model.to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_model_path)

# Set the padding token to be the same as the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

# Specify that padding should be added to the right side of the sequences
tokenizer.padding_side = "right"

# Enable attention cache during inference
tuned_model.config.use_cache = True

Now we load dataset A and generate the test set using same random seed as before.

In [26]:
test_A = pd.read_csv("deliverables/df_csv/test_A.csv", encoding='utf-8')
print(f"items in test test are:{len(test_A)}")

items in test test are:200


In [27]:
def get_output_tuned_model(prompt):
  inputs = tokenizer (prompt, return_tensors="pt")
  inputs = inputs.to(tuned_model.device)

  output_tokens = tuned_model.generate(
    inputs.input_ids,
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
)[0]
  output = tokenizer.decode(output_tokens, skip_special_tokens=True)
  return output

In [28]:
from tqdm import tqdm

translation_list=[]
for row in tqdm(range(len(test_A)),desc="Translating"):

  my_prompt = f"""German Sentence:{test_A.iloc[row, 0].strip()}.
French Translation:
"""
  translation = get_output_tuned_model(my_prompt)
  translation = translation.replace(my_prompt, "")
  translation_list.append(translation)
print(f'collected translations for {len(translation_list)} items.')

Translating: 100%|██████████| 200/200 [06:39<00:00,  2.00s/it]

collected translations for 200 items.


In [29]:
with open('translations_test_C_qwen.txt', 'w') as f:
    for item in translation_list:
        f.write(f"{item}\n")

In [30]:
import evaluate         ##metric for evaulation, HF library
def calculate_bleu_score(prediction, reference):


  #reference = [[reference.lower().split()]]
  #prediction = [prediction.lower().split()]
  prediction = [prediction]
  reference = [[reference]]

  bleu = evaluate.load("bleu")
  score = bleu.compute(predictions= prediction,references=reference)

  return score

In [31]:
from tqdm import tqdm

def get_evaluation_df(translation_list, actual_df):
  """
  translation_list is a list of translations (has just the target language sentences) from the llm to be evaluated
  actual_df : a dataframe containing the actual sentence and its translation
  """
  evaluation =[]
  evaluation_df = pd.DataFrame(columns=[
    'Actual', 
    'Reference', 
    'Prediction', 
    'BLEU Score', 
    'Precisions', 
    'Brevity Penalty', 
    'Length Ratio', 
    'Translation Length', 
    'Reference Length'
])
  for i in tqdm(range(len(translation_list))):
    actual = actual_df.iloc[i,0]
    reference = actual_df.iloc[i,1]
    prediction = translation_list[i]
    score = calculate_bleu_score(prediction, reference)

    evaluation_data = {
        'Actual': actual,
        'Reference': reference,
        'Prediction': prediction,
        'BLEU Score': score['bleu'],
        'Precisions': score['precisions'],
        'Brevity Penalty': score['brevity_penalty'],
        'Length Ratio': score['length_ratio'],
        'Translation Length': score['translation_length'],
        'Reference Length': score['reference_length']
    }
    evaluation.append(evaluation_data)
    
    # Concatenate all evaluation data into the DataFrame
    evaluation_df_concat = pd.concat([evaluation_df, pd.DataFrame(evaluation)], ignore_index=True)

  return evaluation_df_concat


In [32]:
result_df = get_evaluation_df(translation_list, test_A )

  0%|          | 0/200 [00:00<?, ?it/s]/tmp/ipykernel_75540/2784772388.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  evaluation_df_concat = pd.concat([evaluation_df, pd.DataFrame(evaluation)], ignore_index=True)
100%|██████████| 200/200 [01:31<00:00,  2.18it/s]


In [33]:
result_df.shape

(200, 9)

In [34]:
print(f'average bleu score is {result_df["BLEU Score"].mean()}')

average bleu score is 0.0056682964741560996


In [35]:
result_df.to_csv('results_df_C_step10.csv', encoding='utf-8')